In [1]:
import pandas as pd
from math import ceil
from random import randint


In [2]:
def generate_sales(df):
    if 'level_0' in df.columns:
        df.drop('level_0', axis=1, inplace=True)
    df.reset_index(inplace=True)
    df['sales']=df.apply(lambda row:0,axis=1)
    df.loc[0,'sales']=200
    df.loc[1,'sales'] = 250
    for i in range(2, len(df)):
        choice = randint(1,10)
        dif = df.loc[i-1,'score'] - df.loc[i-2, 'score']
        if choice%2==0 or dif < 0:
            new_dif = ceil(((dif - (-2))/(2-(-2)) ) * (500-30) + 30 )
            if dif > 0:
                sales = new_dif
            else:
                sales = new_dif
        else:
            sales= randint(30,501)
        df.loc[i,'sales'] = sales
    return df

In [3]:
def generate_sales_multi_product(df):
    temp = {}
    for i in df.columns: 
        temp[i] = []

    df_new = pd.DataFrame(temp)
    grouped_products = df.groupby(["id"])
    for _, product in grouped_products:
        product = generate_sales(product)
        temp_df = pd.concat([df_new, product])
        if 'level_0' in temp_df.columns:
            temp_df.drop(['level_0'], axis=1, inplace=True)
        df_new = temp_df.reset_index()
    
    return df_new

In [4]:
def generate_price(df:pd.DataFrame, init_price = 20000):
    if 'level_0' in df.columns:
        df.drop('level_0', axis=1, inplace=True)
    df.reset_index(inplace=True)
    if 'price' not in df.columns: 
        df['price']=df.apply(lambda row:0,axis=1)
        df.loc[0,'price']=init_price
        df.loc[1,'price'] = init_price
    init_price = df.loc[0,'price']

    price_range = [(init_price - (0.15 * init_price)) ,(init_price + (0.15 * init_price ))]
    maxDif = (price_range[1] - price_range[0] ) / 10
    for i in range(2, len(df)):
        score_dif = df.loc[i-1,'score'] - df.loc[i-2,'score']
        sales = df.loc[i-1,'sales']
        price = df.loc[i-1, 'price']
        if score_dif > 0 and sales < df['sales'].iloc[i-2]:
            df.loc[i,'price'] = price
        else:
            new_price1 =ceil( ((score_dif + 1)/2) * (price_range[1] - price_range[0]) + price_range[0])
            new_price2 = ceil(  ((sales - 30)/470) *  (price_range[1] - price_range[0]) + price_range[0] )

            new_price = ceil((new_price1 + new_price2) / 2)
            priceDif = new_price - price
            if ( abs(priceDif)  > maxDif ):
                if priceDif < 0:
                    new_price = price - maxDif
                else:
                    new_price = price + maxDif
            df.loc[i,'price'] = new_price

    return df
               

In [5]:
def generate_price_multi_product(df):
    temp = {}
    for i in df.columns: 
        temp[i] = []

    new_df = pd.DataFrame(temp)

    for _, group in df.groupby(['id']):
        group = generate_price(group)
        new_df = pd.concat([new_df, group])
    return new_df


In [37]:
df=pd.read_csv("scraped_dataset/laptop3_processed.csv")

In [38]:
df  = generate_sales_multi_product(df)
df = generate_price_multi_product(df)

In [39]:
index_cols = ['level_0', 'index']
for col in index_cols:
    if col in df:
        df.drop([col], axis=1, inplace=True)

In [40]:
df.to_csv('scraped_dataset/laptop3_priced.csv', index=False)

In [32]:
df.head()

,id,week,score,price,sales
0,0.0,1.0,1.0,35990.0,200.0
1,0.0,2.0,1.0,35990.0,250.0
2,0.0,3.0,-1.0,35818.0,265.0
3,0.0,4.0,1.0,34738.3,30.0
4,0.0,5.0,1.0,34738.3,476.0
